# Obtain internet resources

Beyond communication, people have been utilizing the internet for content creation and consumption. There are a few ways to obtain these contents.

## Web scraping (HTML parsing)

Technically there are [many techniques](https://en.wikipedia.org/wiki/Web_scraping#Techniques) under the category of web scraping. This section will focus solely on the HTML (short for Hypertext Markup Language) parsing technique, which is to automate what humans would do to ingest information from a website manually.

A semantic understanding of the language enables the foundation of the HTML parsing technique. Regardless of how complex and dynamic the processes are behind the website (or web app), the eventual content is delivered as HTML, plus CSS (short for Cascading Style Sheet) for styling, and usually JavaScript for interactivities.

In Python, we can leverage the open-source framework [Scrapy](https://scrapy.org/) to crawl and scrape data from websites.

### A Canadian University Spider

In [1]:
import json

from scrapy import Spider


# our first "Spider" (that crawls the designated website for us)
class UniversitySpider(Spider):

    name = 'University Spider'
    start_urls = ['https://en.wikipedia.org/wiki/List_of_universities_in_Canada']
    
    custom_settings = {
        'ITEM_PIPELINES': { 'item_pipeline.ItemPipeline': 300 },  # from item_pipeline.py
        'LOG_LEVEL': 'ERROR',
    }

    def parse(self, response):
        rows = response.css('table.wikitable > tbody > tr')

        for row in rows:
            school = row.xpath('td[1]')

            if school.css('a ::text'):
                yield response.follow(school.css('a')[0], self.school_parser)

    def school_parser(self, response):
        school_info = {}
        school_info['name'] = response.css('h1.firstHeading ::text').get()

        school_info['lat'] = response.css('span.latitude ::text').get()
        school_info['lng'] = response.css('span.longitude ::text').get()

        rows = response.css('table.infobox > tbody > tr')
        # fuzzy search
        for row in rows:
            header = row.css('th ::text').get()
            if header:
                school_info[header] = row.css('td ::text').get()

        yield school_info

To make a scraping script, we write a `class` by extending the `scrapy.Spider` base class that abstracts away the underlying process so we can focus on specifics such as:
* The starting website URLs for the "Spider" to crawl.
* Rules-based on HTML and CSS selectors to:
    * Next level links to follow.
    * Parse and pick out essential information we want to collect.

Besides the extension of a base class, another new concept is `yield`, which involves the Python generator mechanism. A generator allows a function (or method) to behave like an iterator, which we can conceptualize as an efficient way of interacting with something like a `list`. You can read more about it on its [Python Wiki entry](https://wiki.python.org/moin/Generators). In short, `yield` behaves very much like `return`, but it may keep going until the iterative or concurrent logic surrounding it exhausts all possible inputs.

In [2]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess()
process.crawl(UniversitySpider)
process.start()

2021-03-04 14:40:04 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-04 14:40:04 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (default, May  5 2020, 15:52:07) - [Clang 11.0.0 (clang-1100.0.33.17)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform macOS-10.16-x86_64-i386-64bit
2021-03-04 14:40:04 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-04 14:40:04 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 'ERROR'}


In [3]:
import pandas as pd

# load university data into a Pandas DataFrame
df = pd.read_json('./universities.json')
df

,name,lat,lng,Former names,Type,Established,President,Academic staff,Administrative staff,Students,...,Opened,Tenants,Commandant,Call signs,Athletic teams,Principal and Vice-Chancellor,Tag line,Vice-president,Faculty,Public transit
0,Alberta University of the Arts,51°03′43″N,114°05′29″W,\n,Public,1926,Dr. Daniel Doz,145,95,"1,323",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,University of Northern British Columbia,53°53′14.40″N,122°48′49.40″W,NaN,Public university,1990,Geoffrey Payne (Interim),NaN,NaN,3570 (2019/2020),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,University of the Fraser Valley,49°1′43.69″N,122°17′8.44″W,NaN,Public university,1974,Joanne MacLean,710,NaN,"15,176 (c. 2012)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,University of Victoria,48°27′48″N,123°18′42″W,Victoria College,Public university,"July 1, 1963",Kevin Hall,914 faculty,"5,251 employees","21,696",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,University College of the North,53°49′11″N,101°14′16″W,Keewatin Community College (1966-2004),University college,"July 1, 2004 as University College of the North",Doug Lauvstad,NaN,Approximately 400,"Approximately 2,400",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Capilano University,49°19′07″N,123°01′10″W,NaN,Public,1968,Paul Dangerfield,NaN,NaN,"12,700 (2019)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,University of New Brunswick,45°56′44″N,66°38′27″W,Academy of Liberal Arts and Sciences (1785–1800),Public,1785,Paul Mazerolle,747 FTE,NaN,"8,303",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Athabasca University,54°43′20.63″N,113°18′12.19″W,NaN,Public university,1970,Neil Fassina,NaN,NaN,"40,722",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,233",NaN
91,Mount Royal University,51°0′49.09″N,114°8′0.54″W,NaN,Public,1910,Tim Rahilly,740,NaN,"14,258",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# convert DMS (Degrees-Minutes-Seconds) format to pure numerical decimal point format
# decimal = (degrees + minutes / 60 + seconds  / (60 * 60) * (-1 if S or W else 1)
def decimal_coord(dms):
    try:
        degs, parts = dms.split('°')
        mins, parts = parts.split('′')
        try:
            secs, sign = parts.split('″')
        except:
            sign = parts
        sign = -1 if sign in ['S', 'W'] else 1
        return (float(degs) + float(mins) / 60 + float(secs) / (60 * 60)) * sign
    except:
        return 0


# apply the mutation to the DataFrame
df['lat'] = df['lat'].map(decimal_coord)
df['lng'] = df['lng'].map(decimal_coord)
df

,name,lat,lng,Former names,Type,Established,President,Academic staff,Administrative staff,Students,...,Opened,Tenants,Commandant,Call signs,Athletic teams,Principal and Vice-Chancellor,Tag line,Vice-president,Faculty,Public transit
0,Alberta University of the Arts,51.061944,-114.091389,\n,Public,1926,Dr. Daniel Doz,145,95,"1,323",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,University of Northern British Columbia,53.887333,-122.813722,NaN,Public university,1990,Geoffrey Payne (Interim),NaN,NaN,3570 (2019/2020),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,University of the Fraser Valley,49.028803,-122.285678,NaN,Public university,1974,Joanne MacLean,710,NaN,"15,176 (c. 2012)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,University of Victoria,48.463333,-123.311667,Victoria College,Public university,"July 1, 1963",Kevin Hall,914 faculty,"5,251 employees","21,696",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,University College of the North,53.819722,-101.237778,Keewatin Community College (1966-2004),University college,"July 1, 2004 as University College of the North",Doug Lauvstad,NaN,Approximately 400,"Approximately 2,400",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Capilano University,49.318611,-123.019444,NaN,Public,1968,Paul Dangerfield,NaN,NaN,"12,700 (2019)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,University of New Brunswick,45.945556,-66.640833,Academy of Liberal Arts and Sciences (1785–1800),Public,1785,Paul Mazerolle,747 FTE,NaN,"8,303",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Athabasca University,54.722397,-113.303386,NaN,Public university,1970,Neil Fassina,NaN,NaN,"40,722",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,233",NaN
91,Mount Royal University,51.013636,-114.133483,NaN,Public,1910,Tim Rahilly,740,NaN,"14,258",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# plot it on a map visualization
import folium

m = folium.Map()

def plot(map):

    def fn(row):
        return folium.CircleMarker(
            location=[row['lat'], row['lng']],
            radius=10,
            fill_color='blue',
            popup=row['name'],
        ).add_to(map)

    return fn

# functionally emulate what can be done iteratively
df.apply(plot(m), axis=1)

m

## Web APIs

API stands for Application Programming Interface. Such interfaces expose the underlying abstractions of a computer system or software stacks in a controlled manner.

Web APIs involve the abstractions made available through web servers. The most widely adopted protocol to expose web APIs is HTTP (Hypertext Transfer Protocol).

Technically, web API consumption is a more efficient means to perform web scraping. Instead of parsing the output HTML intended for humans, web APIs usually respond with raw data that is more suitable for direct programming.

We will use the [HH (household) Segments API](https://docs.eqworks.io) from our LOCUS product for our examples below.

In [30]:
import requests

# obtain a valid EQ API JWT of your own
JWT = '<EQ API JWT>'
headers = {'eq-api-jwt': JWT}

coords = [
    '49.8435,-97.1082',
    '49.1623,-122.016',
    '45.4141,-72.74',
    '45.3989,-73.5515',
    '42.9885,-81.227',
    '45.4118,-75.7146',
    '43.9153,-78.8869',
    '46.5012,-81.0069',
    '43.5442,-79.6032',
    '49.19,-122.8573',
    '43.7608,-79.5757',
    '43.8239,-79.0855',
    '43.7561,-79.4046',
    '43.788,-79.4464',
]
params = {'coords[]': coords}

req = requests.get('https://api.locus.place/prod/segment/hh-segments', headers=headers, params=params)
data = req.json()
hh_df = pd.DataFrame(data)
hh_df

,lat,long,segments
0,49.8435,-97.1082,{'cannabis_user': 0.06400000000000006}
1,49.1623,-122.0160,{'cannabis_user': 0.06400000000000006}
2,45.4141,-72.7400,{'cannabis_user': 0.08000000000000007}
3,45.3989,-73.5515,{'cannabis_user': 0.10240000000000005}
4,42.9885,-81.2270,{'school_students': 0.019199999999999995}
5,45.4118,-75.7146,"{'school_students': 0.03839999999999999, 'univ..."
6,43.9153,-78.8869,"{'school_students': 0.272, 'movie_goer': 0.328..."
7,46.5012,-81.0069,"{'university_student': 0.02400000000000002, 's..."
8,43.5442,-79.6032,{'school_students': 0.6320000000000001}
9,49.1900,-122.8573,{'cannabis_user': 0.19200000000000006}


In [35]:
# pick out primary segment (segment with highest score)
def find_primary(segments):
    score = 0
    primary = None
    for k, v in segments.items():
        if v > score:
            primary = k
            score = v

    return primary

hh_df['primary'] = hh_df['segments'].map(find_primary)

# get primary segment score
hh_df['score'] = hh_df.apply(lambda row: row['segments'][row['primary']], axis=1)

hh_df

,lat,long,segments,primary,score
0,49.8435,-97.1082,{'cannabis_user': 0.06400000000000006},cannabis_user,0.0640
1,49.1623,-122.0160,{'cannabis_user': 0.06400000000000006},cannabis_user,0.0640
2,45.4141,-72.7400,{'cannabis_user': 0.08000000000000007},cannabis_user,0.0800
3,45.3989,-73.5515,{'cannabis_user': 0.10240000000000005},cannabis_user,0.1024
4,42.9885,-81.2270,{'school_students': 0.019199999999999995},school_students,0.0192
5,45.4118,-75.7146,"{'school_students': 0.03839999999999999, 'univ...",university_student,0.0832
6,43.9153,-78.8869,"{'school_students': 0.272, 'movie_goer': 0.328...",movie_goer,0.3280
7,46.5012,-81.0069,"{'university_student': 0.02400000000000002, 's...",school_students,0.4320
8,43.5442,-79.6032,{'school_students': 0.6320000000000001},school_students,0.6320
9,49.1900,-122.8573,{'cannabis_user': 0.19200000000000006},cannabis_user,0.1920


In [97]:
# map 'em up
hh_m = folium.Map(location=[43.651890, -79.381706], zoom_start=4)


def get_color(seg):
    return {
        'cannabis_user': 'green',
        'school_students': 'blue',
        'university_student': 'red',
        'movie_goer': 'yellow',
        'frequent_traveler': 'purple',
    }[seg] or 'grey'


def format_popup(row):
    s = f"<h4>{' '.join(row['primary'].split('_'))}</h4>"
    s += f"<p>Coord: {row['lat']}, {row['long']}</p>"
    if len(row['segments']) > 1:
        s += '<ul>'
        for k, v in row['segments'].items():
            s += f'<li>{k}: {v}</li>'
        s += '</ul>'
    return folium.Popup(s, max_width=500)


def plot(map):

    def fn(row):
        color = get_color(row['primary'])
        return folium.CircleMarker(
            location=[row['lat'], row['long']],
            color=color,  # use a pre-determined colors
            fill_color=color,
            fill_opacity=row['score'], # scale opacity against primary segment score
            popup=format_popup(row),
        ).add_to(map)

    return fn

# functionally emulate what can be done iteratively
hh_df.apply(plot(hh_m), axis=1)

hh_m